In [2]:
# This block is to install ase, tk for the GUI, and ipywidgets-jsonschema
import sys

print(sys.executable)
# !{sys.executable} -m pip install ase
# !{sys.executable} -m pip install tk
import tkinter

print(tkinter.Tcl().eval("info patchlevel"))
# !{sys.executable} -m pip install ipywidgets-jsonschema


/Users/mat3ra/.pyenv/versions/3.11.4/bin/python
8.6.13


In [3]:
# data that comes from JS, here are some examples
poscars = {
    "Ni": """Ni - primitive
1.0
   2.130422000	   0.000000000	   1.230000000
   0.710140800	   2.008582000	   1.230000000
   0.000000000	   0.000000000	   2.460000000
Ni
1
Selective dynamics
direct
   0.000000000    0.000000000    0.000000000 T T T Ni
""",
    "Cu": """Cu4
1.0
   3.5774306715697510    0.0000000000000000    0.0000000000000002
   0.0000000000000006    3.5774306715697510    0.0000000000000002
   0.0000000000000000    0.0000000000000000    3.5774306715697510
Cu
4
direct
   0.0000000000000000    0.0000000000000000    0.0000000000000000 Cu
   0.0000000000000000    0.5000000000000000    0.5000000000000000 Cu
   0.5000000000000000    0.0000000000000000    0.5000000000000000 Cu
   0.5000000000000000    0.5000000000000000    0.0000000000000000 Cu
""",
    "Au": """Au4
1.0
   4.1712885314747270    0.0000000000000000    0.0000000000000003
   0.0000000000000007    4.1712885314747270    0.0000000000000003
   0.0000000000000000    0.0000000000000000    4.1712885314747270
Au
4
direct
   0.0000000000000000    0.0000000000000000    0.0000000000000000 Au
   0.0000000000000000    0.5000000000000000    0.5000000000000000 Au
   0.5000000000000000    0.0000000000000000    0.5000000000000000 Au
   0.5000000000000000    0.5000000000000000    0.0000000000000000 Au
""",
    "SiC": """Si4 C4
1.0
   4.3539932475828609    0.0000000000000000    0.0000000000000003
   0.0000000000000007    4.3539932475828609    0.0000000000000003
   0.0000000000000000    0.0000000000000000    4.3539932475828609
Si C
4 4
direct
   0.7500000000000000    0.2500000000000000    0.7500000000000000 Si4+
   0.7500000000000000    0.7500000000000000    0.2500000000000000 Si4+
   0.2500000000000000    0.2500000000000000    0.2500000000000000 Si4+
   0.2500000000000000    0.7500000000000000    0.7500000000000000 Si4+
   0.0000000000000000    0.0000000000000000    0.0000000000000000 C4-
   0.0000000000000000    0.5000000000000000    0.5000000000000000 C4-
   0.5000000000000000    0.0000000000000000    0.5000000000000000 C4-
   0.5000000000000000    0.5000000000000000    0.0000000000000000 C4-
""",
    "Graphene": """Graphene
1.0
   2.467291000	   0.000000000	   0.000000000
  -1.233645000	   2.136737000	   0.000000000
   0.000000000	   0.000000000	   7.803074000
C
2
direct
   0.000000000    0.000000000    0.000000000  C
   0.333333000    0.666667000    0.000000000  C
""",
}


In [9]:
import io
from ase.build import surface, make_supercell
from ase.io import read, write
from ase import Atoms
import numpy as np
from ase.visualize import view

# Define a function to read POSCAR content and return an Atoms object
def poscar_to_atoms(poscar_content):
    poscar_io = io.StringIO(poscar_content)
    atoms = read(poscar_io, format="vasp")
    return atoms

# Define a function to create a surface and supercell
def prepare_surface_and_supercell(atoms, miller_indices, layers, vacuum, superlattice_matrix):
    # Create the surface
    surface_atoms = surface(atoms, miller_indices, layers, vacuum)
    # Create the supercell
    supercell_atoms = make_supercell(surface_atoms, superlattice_matrix)
    return supercell_atoms

# Define a function to calculate the scaling factors for lattice vectors
def calculate_scaling_factors(substrate_cell, layer_cell):
    # Calculate the norm of the a and b vectors for both substrate and layer
    substrate_a_norm = np.linalg.norm(substrate_cell[0])
    substrate_b_norm = np.linalg.norm(substrate_cell[1])
    layer_a_norm = np.linalg.norm(layer_cell[0])
    layer_b_norm = np.linalg.norm(layer_cell[1])
    
    # Calculate the scaling factors for a and b
    scale_a = substrate_a_norm / layer_a_norm
    scale_b = substrate_b_norm / layer_b_norm
    
    return scale_a, scale_b

# Define a function to apply scaling to the layer's lattice vectors
def scale_layer(layer_atoms, scale_a, scale_b):
    # Scale the a and b vectors by the calculated factors
    layer_cell = layer_atoms.get_cell()
    layer_cell[0] *= scale_a
    layer_cell[1] *= scale_b
    layer_atoms.set_cell(layer_cell, scale_atoms=True)
    
    return layer_atoms

# Define a function to calculate strain on the layer
def calculate_strain(original_layer_cell, scaled_layer_cell):
    # Calculate the original and scaled norms
    original_a_norm = np.linalg.norm(original_layer_cell[0])
    original_b_norm = np.linalg.norm(original_layer_cell[1])
    scaled_a_norm = np.linalg.norm(scaled_layer_cell[0])
    scaled_b_norm = np.linalg.norm(scaled_layer_cell[1])
    
    # Calculate the strain along a and b
    strain_a = (scaled_a_norm - original_a_norm) / original_a_norm
    strain_b = (scaled_b_norm - original_b_norm) / original_b_norm
    
    return strain_a, strain_b

# Define the main function to create the interface
def create_interface(substrate_poscar, layer_poscar, surface_params, superlattice_matrices, z_offset):
    # Convert POSCAR content to Atoms objects
    substrate_atoms = poscar_to_atoms(substrate_poscar)
    layer_atoms = poscar_to_atoms(layer_poscar)
    
    # Prepare the substrate surface and supercell
    substrate_supercell = prepare_surface_and_supercell(
        substrate_atoms, 
        (surface_params["miller"]["h"], surface_params["miller"]["k"], surface_params["miller"]["l"]),
        surface_params["number_of_layers"], 
        surface_params["vacuum"], 
        superlattice_matrices["slab"]
    )
    
    # Prepare the layer supercell
    layer_supercell = prepare_surface_and_supercell(
        layer_atoms, 
        (0, 0, 1),  # Assuming the layer is already a 2D material, just a single layer
        1,  # Only one layer needed for a 2D material
        0,  # No additional vacuum needed, it's already in the POSCAR
        superlattice_matrices["layer"]
    )
    view(layer_supercell)

    # Calculate scaling factors
    scale_a, scale_b = calculate_scaling_factors(substrate_supercell.get_cell(), layer_supercell.get_cell())
    
    # Save the original layer cell for strain calculations
    original_layer_cell = layer_supercell.get_cell().copy()
    
    # Scale the layer
    scaled_layer_supercell = scale_layer(layer_supercell, scale_a, scale_b)
    
    # Calculate strain on the layer
    strain_a, strain_b = calculate_strain(original_layer_cell, scaled_layer_supercell.get_cell())
    print("Strain along a:", strain_a)
    print("Strain along b:", strain_b)


    # Adjust Z position based on the Z offset
    z_max_substrate = max(substrate_supercell.positions[:, 2])
    layer_supercell.positions[:, 2] += z_max_substrate + z_offset
    
    # Combine substrate and layer into one Atoms object
    interface = substrate_supercell + layer_supercell
    
    return interface

# Example usage
substrate_poscar = poscars["Ni"]
layer_poscar = poscars["Graphene"]
surface_params = {
    "miller": {"h": 1, "k": 1, "l": 1},
    "number_of_layers": 3,
    "vacuum": 10,
}
superlattice_matrices = {
    "slab": [[1, 0, 0], [0, 1, 0], [0, 0, 1]],  # Example superlattice matrix for substrate
    "layer": [[1, 0, 0], [0, 1, 0], [0, 0, 1]],  # Example superlattice matrix for layer
}
z_offset = 3.0  # Example Z offset

# Create the interface
interface_atoms = create_interface(substrate_poscar, layer_poscar, surface_params, superlattice_matrices, z_offset)

# Optional: Save the interface to a file
write("interface_POSCAR.vasp", interface_atoms, format="vasp")

# Visualize the interface


view(interface_atoms*(3,3,1))
print(interface_atoms)


Strain along a: -0.0029552360133994426
Strain along b: -0.0029549398929407155
Atoms(symbols='Ni3C2', pbc=[True, True, False], cell=[[2.4599995727812636, 0.0, 0.0], [1.2300000442669368, 2.1304228448014664, 0.0], [0.0, 0.0, 24.017163104672278]])
